In [ ]:
println(s"Current spark version is ${spark.version}")

In [ ]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame


In [ ]:
val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))


In [ ]:
val inputStreamPath = "/home/jovyan/data/events-stream"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

In [ ]:
val modelPath = "/home/jovyan/models/spark-ml-model"
val model = PipelineModel.load(modelPath)


In [ ]:
val predictionDF = model
    .transform(inputDF)
    .select($"tweet", getProbability($"probability").alias("clean_probability"), $"prediction")


In [ ]:
val output1 = predictionDF
    .writeStream
    .foreachBatch { (batchDF: DataFrame, batchId: Long) => batchDF.show() }
    .start()

output1.awaitTermination(10000)

In [ ]:
output1.stop()

In [ ]:
val output2 = predictionDF
    .withColumn("timestamp", current_timestamp)
    .withWatermark("timestamp", "10 seconds")
    .groupBy(window($"timestamp", "10 seconds"), $"prediction")
    .count
    .writeStream
    .outputMode("append")
    .format("console")
    .option("truncate", "false")
    .start()

output2.awaitTermination(100000)

In [ ]:
output2.stop()